In [ ]:
import pandas as pd
from prophet import Prophet
# Load the data
file_path = '../../Data/Scripts/dataProcesing/TimeSort.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Combine Date and Time Period into a single Datetime column if not already done
# data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time Period'])

# Check for NaN values in 'Datetime' column
if data['Datetime'].isnull().any():
    # Option 1: Remove rows with NaN values
    # data = data.dropna(subset=['Datetime'])

    # Option 2: Impute missing values (forward fill, backward fill, or interpolation)
    data['Datetime'].fillna(method='ffill', inplace=True)

    # Add your chosen method of handling NaN values here

# Rename the relevant columns for Prophet
data = data.rename(columns={'Datetime': 'ds', 'Min. per HM': 'y'})

# Convert 'ds' to datetime format
data['ds'] = pd.to_datetime(data['ds'])

# Initialize and fit the Prophet model
model = Prophet()
model.fit(data)

# Making future predictions
# As the dataset contains hourly data, we will predict for the next 48 hours (2 days)
future = model.make_future_dataframe(periods=48, freq='H')
forecast = model.predict(future)

# Displaying the last few rows to see the predictions
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load data
file_path = '../../Data/Scripts/dataProcesing/TimeSort.csv'   # Replace with your file path
data = pd.read_csv(file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Setting the 'Datetime' as the index
data.set_index('Datetime', inplace=True)

# Resampling the data to daily frequency (or another frequency if more suitable)
data_daily = data['Min. per HM'].resample('D').mean()

# Check for stationarity
result = adfuller(data_daily.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# Apply differencing based on stationarity test
data_to_model = data_daily
if result[1] > 0.05:
    data_to_model = data_daily.diff().dropna()

# Model Fitting
model = ARIMA(data_to_model, order=(5,1,0))  # Adjust the order parameters as needed
model_fit = model.fit()

# Summary of the model
print(model_fit.summary())

# Predictions
plt.figure(figsize=(10, 6))
plt.plot(data_to_model, label='Original')
plt.plot(model_fit.fittedvalues, color='red', label='Fitted Values')
plt.legend()
plt.show()

In [ ]:
data = pd.read_csv(file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Setting the 'Datetime' as the index
data.set_index('Datetime', inplace=True)

# Resampling the data to daily frequency
data_daily = data['Min. per HM'].resample('D').mean()

# Apply differencing if data is not stationary
result = adfuller(data_daily.dropna())
data_to_model = data_daily.diff().dropna() if result[1] > 0.05 else data_daily

# Model Fitting
model = ARIMA(data_to_model, order=(5,1,0))  # Adjust the order parameters as needed
model_fit = model.fit()

# Forecasting Future Dates
num_days = 30  # Number of days to predict into the future
forecast = model_fit.forecast(steps=num_days)

# Creating a date range for the forecasted data
last_date = data_to_model.index[-1]
future_dates = pd.date_range(start=last_date, periods=num_days + 1)[1:]

# Creating a DataFrame for the forecast
forecast_df = pd.DataFrame(forecast, index=future_dates, columns=['Forecast'])

# Plotting the forecasts along with the original data
plt.figure(figsize=(10, 6))
plt.plot(data_to_model, label='Original Data')
plt.plot(forecast_df, color='red', label='Forecast')
plt.legend()
plt.show()

In [ ]:
data = pd.read_csv(file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Drop rows with NaN values in 'Min. per HM' or 'Datetime'
data = data.dropna(subset=['Datetime', 'Min. per HM'])

# Preparing data for Prophet
prophet_data = data.rename(columns={'Datetime': 'ds', 'Min. per HM': 'y'})

# Initialize the Prophet model
model = Prophet()

# Fit the model
model.fit(prophet_data)

# Create a dataframe for future predictions
future = model.make_future_dataframe(periods=30)  # Predicting 30 days into the future

# Forecasting
forecast = model.predict(future)

# Plotting the forecast
fig1 = model.plot(forecast)
plt.show()